In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from nltk.tag import StanfordPOSTagger
from nltk import word_tokenize
from nltk.internals import find_jars_within_path
#import nltk
#nltk.download('punkt')

In [2]:
root = "/Users/ashleyroakes/Desktop/"
mim_root = root + "mimic-iii-clinical-database-1.4/"

# Data Pre-processing
## Read in Discharge Notes

In [97]:
notes = mim_root + "NOTEEVENTS.csv.gz"

notes_df = pd.read_csv(notes, compression='gzip', error_bad_lines=False, 
                       usecols = ['SUBJECT_ID', 'HADM_ID', 'CATEGORY', 'DESCRIPTION','TEXT'])\
                      .query("CATEGORY == 'Discharge summary'")\
                      .query("DESCRIPTION == 'Report'")

# Should be 55,177 records
print("Number of discharge summaries: ", + len(notes_df))

notes_df.head()

Number of discharge summaries:  55177


,SUBJECT_ID,HADM_ID,CATEGORY,DESCRIPTION,TEXT
0,22532,167853.0,Discharge summary,Report,Admission Date: [**2151-7-16**] Dischar...
1,13702,107527.0,Discharge summary,Report,Admission Date: [**2118-6-2**] Discharg...
2,13702,167118.0,Discharge summary,Report,Admission Date: [**2119-5-4**] D...
3,13702,196489.0,Discharge summary,Report,Admission Date: [**2124-7-21**] ...
4,26880,135453.0,Discharge summary,Report,Admission Date: [**2162-3-3**] D...


## Read in Patient Diagnoses

In [98]:
diag = mim_root + "DIAGNOSES_ICD.csv.gz"

diag_df = pd.read_csv(diag, compression='gzip', error_bad_lines=False)\
                    .dropna()\
                    .groupby('HADM_ID')['ICD9_CODE']\
                    .unique()\
                    .reset_index()
diag_df.head()

,HADM_ID,ICD9_CODE
0,100001,"[25013, 3371, 5849, 5780, V5867, 25063, 5363, ..."
1,100003,"[53100, 2851, 07054, 5715, 45621, 53789, 4019,..."
2,100006,"[49320, 51881, 486, 20300, 2761, 7850, 3090, V..."
3,100007,"[56081, 5570, 9973, 486, 4019]"
4,100009,"[41401, 99604, 4142, 25000, 27800, V8535, 4148..."


## Read in ICD9 Dictionary

In [99]:
icd = mim_root + "D_ICD_DIAGNOSES.csv.gz"

icd_df = pd.read_csv(icd, compression='gzip', error_bad_lines=False)

icd_df.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


## Merge datasets by HADM_ID

In [100]:
df = pd.merge(diag_df, notes_df, on='HADM_ID', how='inner')

# Should be 55177-5 = 55172
len(df)

55172

## Split data in train, valid, and test sets
### training (38,588 records, 69.9%), validation (5536 records, 10.0%) and testing (11,048 records, 20.0%) folds

In [101]:
# Random split
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)

## Substitute special sequences,  Filter HoPI sections, & Tokenize

### Identify HOPI sections & Substitute Special Sequences: These special sequences were identified and replaced by the first token in the sequence, e.g. "[**Hospital1 18**]" was replaced by ‘Hospital1’.

In [623]:
def process_notes(st):
    s  = "History of Present Illness"
    s1 = "HISTORY OF PRESENT ILLNESS|HISTORY OF THE PRESENT ILLNESS|REASON FOR ADMISSION|present illness|Present Illness|PRESENT ILLNESS"
    
    match  = re.search(s, st)
    match1 = re.search(s1, st)
    
    if (match is not None) or (match1 is not None):
        if match is not None:
            st = st.split(s, 1)[1]
            e = "\n\n\n"
            n = st.split(e, 1)[0]
    
        elif match1 is not None: 
            st = st.split(match1[0], 1)[1]
            e = re.search(r"\n[\s\w]+:", st)[0]
            n = st.split(e, 1)[0]
            
        rep = re.findall(r"\[\*\*([a-zA-Z0-9]*)", n)
        strt = [m.start() for m in re.finditer("\[\*\*([a-zA-Z0-9]*)", n)]
        end = [m.end() for m in re.finditer("([a-zA-Z0-9]*)\*\*\]", n)]

        for i in range(len(rep)):
            n = n[:strt[i]] + rep[i] + " " + n[end[i] + 1:]
        
    else: 
        n = ''
    return n

In [624]:
df['HOPI'] = df["TEXT"].map(lambda t: process_notes(t))
df = df[df['HOPI'] != ""]

### Number of missing HOPI sections

In [625]:
# Detect history of present illness in text (n = 2641 records without HoPI data)
missing = df[df["HOPI"] == ""].reset_index()

len(missing)

# Should be 2641 records
print("Notes without HOPI sections: " + str(len(missing)))

Notes without HOPI sections: 3620


In [673]:
#Testing out other possible HOPI section headers

#PREOPERATIVE DIAGNOSIS:
#Service:
#HOSPITAL COURSE

ms = missing["TEXT"].map(lambda t: re.search("", t.lower()))
m = missing[~ms.isna()].reset_index()

cap = 138
lw = 15
lwlw = 12
hpi = 2
crs = 951
print(3620-len(m))

#m["TEXT"].to_csv(root + "no hopi.csv")

951


### Initialize Stanford Tokenizer

In [102]:
# Add the jar and model via their path (instead of setting environment variables):
jar = root + 'stanford-postagger-full-2020-11-17/stanford-postagger.jar'
model = root + 'stanford-postagger-full-2020-11-17/models/english-left3words-distsim.tagger'

pos_tagger = StanfordPOSTagger(model, jar, encoding='utf8')

# Add other jars from Stanford directory
stanford_dir = pos_tagger._stanford_jar.rpartition('/')[0]
stanford_jars = find_jars_within_path(stanford_dir)
pos_tagger._stanford_jar = ':'.join(stanford_jars)

In [ ]:
df['tokens'] = df["HOPI"].map(lambda n: word_tokenize(n))

## Truncate at 500 tokens

In [ ]:
# Truncate records with more than 500 tokes (n = 1143)


## Plot a histogram of the Number of tokens in each HoPI document, after data preprocessing.

## Count number of tokens in the training dataset (n = ~92,468 tokens)

In [ ]:
# Count occurence of tokens that are in the training dataset

# Tokens that occur >= 5 times are in the study vocabulary

# Assign a unique integer ID for each token in the study vocabulary 

# Convert each HoPI document to a 1D array of integers using this index


# Document representation

In [ ]:
# Represent clinical notes documnets as TF-IDF representation

In [ ]:
# Mean embedding representation

In [ ]:
# GRU representation

# Label representation